In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import os
import numpy as np
import pandas as pd
from torch import nn

import gymnasium as gym

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from datetime import datetime
from tqdm import tqdm

from inspect import getsource
import torch
import random
from collections import deque

from drl_nav.agent import Agent_PER, Agent_DQN_pixel
from drl_nav.network.heads import LabelizerNet, AuxNet, QNet
from drl_nav.network.bodies import ConvBody

C:\Users\pierr\environment\RL\lib\site-packages\gymnasium\core.py:27: UserWarning: WARN: Gymnasium minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+
  "Gymnasium minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+"


_____

In [ ]:
# number of actions
n_A = brain.vector_action_space_size
vector_state = env_info.vector_observations[0]
visual_state = env_info.visual_observations[0]
n_S = len(visual_state)


In [12]:
env = gym.make('CartPole-v1')


if len(env.observation_space.shape):
    state_size = env.observation_space.shape[0]
else:
    state_size = env.observation_space.shape

print(f"state size: {state_size}")    
print(f"action space: {env.action_space}")

state size: 4
action space: Discrete(2)


In [9]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [6]:
state_size

(4,)

In [7]:
action_size 

()